In [1]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader("C:/Users/abida/Desktop/Books", glob="**/*.docx")
books = loader.load()
len(books)


c:\Users\abida\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


4

In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(books)


In [26]:
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings

vectorstore = Chroma.from_documents(
    documents=all_splits,
    embedding=OllamaEmbeddings(model="llama3", show_progress=True),
    persist_directory="./chroma_db",
)


OllamaEmbeddings: 100%|██████████| 23/23 [01:00<00:00,  2.63s/it]


In [24]:
question = "Who is Zahra?"
docs = vectorstore.similarity_search(question)
docs


OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.34s/it]


[Document(page_content="In Chapter 2, Zahra embarks on her first independent journey to visit Arslan, her brother's best friend, hoping he can provide some information. Though Arslan is reluctant at first, he eventually mentions a woman named Mayaa who may have known Ali better.", metadata={'source': 'C:\\Users\\abida\\Desktop\\Books\\Ibd Naksh - Book 4.docx'}),
 Document(page_content='Here is a more detailed summary of the book "A Boy with Honest Dreams" by Abid Ali Awan:', metadata={'source': 'C:\\Users\\abida\\Desktop\\Books\\A Boy With Honest Dreams - Book 3.docx'}),
 Document(page_content='mentions a woman named Mayaa who may have known Ali better.', metadata={'source': 'C:\\Users\\abida\\Desktop\\Books\\Ibd Naksh - Book 4.docx'}),
 Document(page_content='Here is a detailed summary of the book "IBD NAKSH" by Abid Ali Awan:', metadata={'source': 'C:\\Users\\abida\\Desktop\\Books\\Ibd Naksh - Book 4.docx'})]

In [16]:
from langchain import hub
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

llm = Ollama(model="llama3")

retriever = vectorstore.as_retriever()


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_prompt = hub.pull("rlm/rag-prompt")
qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)


In [20]:
question = "What is the story of 'Gené is Missing' book?"
qa_chain.invoke(question)


OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.39s/it]


'Based on the provided context, "Gené is Missing" appears to be a mystery novel that revolves around uncovering the truth about Gené\'s life and potential involvement in murders. The story follows different perspectives, including Simon trying to prove his innocence, flashbacks of Gené\'s past, and detective Jacob investigating the case.'

In [25]:
question = "Who is Arslan?"
qa_chain.invoke(question)


OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.36s/it]


'Arslan is Zahra\'s brother\'s best friend. He is someone that Zahra visits in the book "A Boy with Honest Dreams" by Abid Ali Awan to gather information about Ali.'